# Check for presence of EC2 instance

In [32]:
import boto3

ec2 = boto3.client('ec2')

response = ec2.describe_instances()

# instance_name = "mlops-kgptalkie"

instance_name = "mlops-1"

instance_id="" 

for resp in response['Reservations']:
    resp = resp['Instances'][0]
    tags = resp.get('Tags', [])
    
    for tag in tags:
        if tag.get("Key", "")=="Name" and tag.get("Value", "")==instance_name:
            instance_id = resp['InstanceId']

if instance_id=="":
    print(f"No instance found with name {instance_name}")
    # raise("Stop here!!!")

instance_id


No instance found with name mlops-1


''

# Create AWS EC2 instance

In [34]:
ec2 = boto3.client('ec2')

ec2.describe_instances()

resp = ec2.create_key_pair(KeyName='key-1')

file = open("creds/key-1.pem", 'w')
file.write(resp['KeyMaterial'])
file.close()

In [35]:
import boto3

ec2 = boto3.client('ec2')

if instance_id == "":
    response = ec2.run_instances(
        ImageId = 'ami-0197c13a4f68c9360',
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.large',
        KeyName='key-1',
        BlockDeviceMappings=[
            {
                "DeviceName": "/dev/xvda",
                'Ebs':{
                    'DeleteOnTermination': True,
                    'VolumeSize': 120
                }
            }
        ]

    )

    instance_id = response['Instances'][0]['InstanceId']

    ec2.create_tags(Resources=[instance_id], Tags=[
        {
            'Key':'Name',
            'Value':instance_name
        }
    ])

else:
    print("Instance is already present")

# Create security group and add rules

In [36]:
group_name = 'dgwalters'

response = ec2.describe_security_groups()

security_group_id = [x['GroupId'] for x in response['SecurityGroups'] if x['GroupName']==group_name]

if security_group_id == []:
    response = ec2.create_security_group(
        GroupName = group_name,
        Description = "Security group for testing"
    )
    security_group_id = response['GroupId']
else:
    security_group_id = security_group_id[0]

security_group_id

'sg-0295a206172b63740'

In [37]:
from botocore.exceptions import ClientError

def update_security_group(group_id, protocol, port, cidr):
    try:
        response = ec2.authorize_security_group_ingress(
            GroupId = group_id,
            IpPermissions=[
                {
                    'IpProtocol': protocol,
                    'FromPort': port,
                    'ToPort': port,
                    'IpRanges': [{'CidrIp': cidr}]
                }
            ]
        )
    except ClientError as e:
        if e.response['Error']['Code']=='InvalidPermission.Duplicate':
            print('This rule is already there')
        else:
            print("an error as occured!")
            print(e)

update_security_group(security_group_id, 'tcp', 22, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 80, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8501, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8502, '0.0.0.0/0')

This rule is already there
This rule is already there
This rule is already there
This rule is already there


In [ ]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[security_group_id])

{'ResponseMetadata': {'RequestId': 'c22d018e-23c4-4ae3-96da-619bf0150244',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c22d018e-23c4-4ae3-96da-619bf0150244',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Fri, 22 Nov 2024 19:31:11 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

: 

In [7]:
# describe IAM role
iam = boto3.client('iam')

role_name = "ec2-s3-full-access"

response = iam.get_role(RoleName=role_name)

role_arn = response['Role']['Arn']

role_arn

# Ensure there is an instance profile with the same name as the role
instance_profile_name = role_name
try:
    iam.get_instance_profile(InstanceProfileName=instance_profile_name)
except iam.exceptions.NoSuchEntityException:
    # Create an instance profile if it doesn't exist
    iam.create_instance_profile(InstanceProfileName=instance_profile_name)
    # Add role to the instance profile
    iam.add_role_to_instance_profile(
        InstanceProfileName=instance_profile_name,
        RoleName=role_name
    )

# Attach the instance profile to the EC2 instance
ec2.associate_iam_instance_profile(
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    InstanceId=instance_id
)

NoSuchEntityException: An error occurred (NoSuchEntity) when calling the GetRole operation: The role with name ec2-s3-full-access cannot be found.

In [ ]:
54.234.218.4
ssh -i "C:\Users\dgwal\.ssh\dgwalters_new.pem" ubuntu@ec2-54-221-102-75.compute-1.amazonaws.com
ssh -i "dgwalters_new.pem" ubuntu@ec2-54-221-102-75.compute-1.amazonaws.com
ssh -i "C:\Users\dgwal\.ssh\dgwalters_new.pem" ubuntu@98.81.217.238
ssh -i "key-1.pem" ubuntu@ec2-54-81-122-139.compute-1.amazonaws.com

instance = mlops-1

In [31]:
# ec2 = boto3.client('ec2')

# ec2.describe_instances()

resp = ec2.create_key_pair(KeyName='dgw1974')

ClientError: An error occurred (InvalidKeyPair.Duplicate) when calling the CreateKeyPair operation: The keypair already exists

In [ ]:
file = open("creds/dgwalters.pem", 'w')
file.write(resp['KeyMaterial'])
file.close()

In [24]:
resp

{'AmiLaunchIndex': 0,
 'ImageId': 'ami-0197c13a4f68c9360',
 'InstanceId': 'i-0c63f60d2d284623d',
 'InstanceType': 't2.large',
 'KeyName': 'dgw1974',
 'LaunchTime': datetime.datetime(2024, 11, 22, 18, 9, 33, tzinfo=tzutc()),
 'Monitoring': {'State': 'disabled'},
 'Placement': {'AvailabilityZone': 'us-east-1c',
  'GroupName': '',
  'Tenancy': 'default'},
 'PrivateDnsName': '',
 'ProductCodes': [],
 'PublicDnsName': '',
 'State': {'Code': 48, 'Name': 'terminated'},
 'StateTransitionReason': 'User initiated (2024-11-22 18:49:16 GMT)',
 'Architecture': 'x86_64',
 'BlockDeviceMappings': [],
 'ClientToken': '65293c87-f67f-46dd-89fa-1795f29df0f4',
 'EbsOptimized': False,
 'EnaSupport': True,
 'Hypervisor': 'xen',
 'NetworkInterfaces': [],
 'RootDeviceName': '/dev/sda1',
 'RootDeviceType': 'ebs',
 'SecurityGroups': [],
 'StateReason': {'Code': 'Client.UserInitiatedShutdown',
  'Message': 'Client.UserInitiatedShutdown: User initiated shutdown'},
 'Tags': [{'Key': 'Name', 'Value': 'mlops-prod'}],